### **Requirements**

In [13]:
!pip install qiskit_nature
!pip install pyscf
!pip install rdkit
!pip install qiskit_aer
!pip install openfermion openfermionpyscf
!pip install scikit-learn

### **Data Preparation**

In [ ]:
url = 'https://raw.githubusercontent.com/TastelessCandyzzz/Quantum_Chem_Hackathon/main/bace.csv'

In [1]:
import pandas as pd
df = pd.read_csv(url,usecols=["mol","Class","MW"])
print(df.head())

                                                 mol  Class         MW
0  O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...      1  431.56979
1  Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...      1  657.81073
2  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...      1  591.74091
3  S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...      1  591.67828
4  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...      1  629.71283


In [2]:
df=df.loc[df["MW"]<200]
del df["MW"]

,mol,Class
192,n1cccc(NCc2ccccc2)c1N,0
194,Fc1ccc(cc1)CC1CC[NH2+]CC1,0
200,n1ccc2c(cccc2)c1N,0
201,n1c2c(ccc1N)cccc2,0
203,O=C1NC(=NC(=C1)CCC)N,0
245,Oc1ccc(cc1CC)CC[NH3+],1
246,Oc1ccc(cc1)CC([NH3+])C,1
247,Oc1ccc(cc1)CC[NH3+],1
1506,Clc1cc2nc([nH]c2cc1)N,0


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np

def smiles_to_fingerprint(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros((n_bits,))

# Apply to all molecules
df['features'] = df['mol'].apply(smiles_to_fingerprint)

[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator
[09:42:09] DEPRECATION WARNING: please use MorganGenerator


In [75]:
X = np.stack(df["features"])  # Convert feature column into a NumPy matrix
y = df['Class']

## **Calculating Qubit Hamiltonian, VQE ground-state energy, Hartree–Fock energy, Correlation energy, HOMO LUMO Gap and Dipole Moment**

In [88]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops, Descriptors

# --- PySCF Imports ---
from pyscf import gto, scf

# --- Qiskit Core Imports ---
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import efficient_su2
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import BackendEstimatorV2

# --- Qiskit Nature Imports ---
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer

# ============================================================
# Helper: Generate 3D geometry from an RDKit Mol object
# ============================================================
def mol_to_geometry(mol: Chem.Mol):
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)
    atoms = mol.GetAtoms()
    conf = mol.GetConformer()
    return "; ".join(
        f"{atom.GetSymbol()} "
        f"{conf.GetAtomPosition(atom.GetIdx()).x:.4f} "
        f"{conf.GetAtomPosition(atom.GetIdx()).y:.4f} "
        f"{conf.GetAtomPosition(atom.GetIdx()).z:.4f}"
        for atom in atoms
    )

# ============================================================
# Step 1: Convert geometry → qubit Hamiltonian
# ============================================================
def molecule_to_qubit_hamiltonian(
    geometry,
    basis="sto3g",
    charge=0,
    spin_multiplicity=1,
    active_electrons=2,
    active_orbitals=2,
):
    driver = PySCFDriver(
        atom=geometry, basis=basis, charge=charge,
        spin=(spin_multiplicity - 1), unit=DistanceUnit.ANGSTROM
    )
    es_problem = driver.run()
    freezer = FreezeCoreTransformer(freeze_core=True)
    transformer = ActiveSpaceTransformer(
        num_electrons=active_electrons, num_spatial_orbitals=active_orbitals
    )
    es_problem = freezer.transform(es_problem)
    es_problem = transformer.transform(es_problem)
    mapper = JordanWignerMapper()
    second_q_hamiltonian = es_problem.second_q_ops()[0]
    return {"qubit_operator": mapper.map(second_q_hamiltonian), "es_problem": es_problem}

# ============================================================
# Step 2: Main function to compute all descriptors
# ============================================================
def compute_quantum_descriptors(
    smiles: str,
    basis: str = "sto3g",
    active_electrons: int or tuple = 2,
    active_orbitals: int = 2,
):
    mol_rdkit = Chem.MolFromSmiles(smiles)
    if mol_rdkit is None:
        raise ValueError("Invalid SMILES string provided.")

    charge = rdmolops.GetFormalCharge(mol_rdkit)
    num_radical_electrons = Descriptors.NumRadicalElectrons(mol_rdkit)
    spin_multiplicity = num_radical_electrons + 1

    print(f"\n[•] Processing molecule: {smiles}")
    print(f"[i] Auto-detected Charge: {charge}, Auto-detected Spin: {spin_multiplicity}")

    geometry = mol_to_geometry(mol_rdkit)
    out = molecule_to_qubit_hamiltonian(
        geometry, basis, charge, spin_multiplicity, active_electrons, active_orbitals
    )
    qubit_op, es_problem = out["qubit_operator"], out["es_problem"]
    hf_energy = es_problem.reference_energy
    print(f"[+] Hartree–Fock energy: {hf_energy:.6f} Hartree")

    backend_sim = AerSimulator()
    estimator = BackendEstimatorV2(backend=backend_sim)
    ansatz = efficient_su2(qubit_op.num_qubits, reps=2, entanglement="linear")
    ansatz = transpile(ansatz, backend=backend_sim)
    optimizer = COBYLA(maxiter=200)
    vqe_solver = VQE(estimator, ansatz, optimizer)
    vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)
    vqe_energy = vqe_result.eigenvalue.real
    print(f"[+] VQE ground-state energy: {vqe_energy:.6f} Hartree")

    # --- PySCF calculations for additional properties ---
    mol_pyscf = gto.Mole(
        atom=geometry, basis=basis, charge=charge, spin=spin_multiplicity - 1
    ).build()
    mf = scf.RHF(mol_pyscf).run() if spin_multiplicity == 1 else scf.UHF(mol_pyscf).run()

    # Using the robust, low-level dipole calculation
    dm = mf.make_rdm1()
    dip_ints = mf.mol.intor('int1e_r', comp=3)
    elec_dip = -np.einsum('xij,ji->x', dip_ints, dm)
    nuc_dip = np.einsum('i,ix->x', mf.mol.atom_charges(), mf.mol.atom_coords())
    dipole_vector = elec_dip + nuc_dip
    dipole_moment = np.linalg.norm(dipole_vector)

    # --- Property Extraction ---
    if es_problem.orbital_energies_b is None:
        num_occupied = es_problem.num_alpha
        homo_energy = es_problem.orbital_energies[num_occupied - 1]
        lumo_energy = es_problem.orbital_energies[num_occupied]
    else:
        alpha_homo = es_problem.orbital_energies[es_problem.num_alpha - 1]
        beta_homo = es_problem.orbital_energies_b[es_problem.num_beta - 1]
        homo_energy = max(alpha_homo, beta_homo)
        lumo_energy = es_problem.orbital_energies[es_problem.num_alpha]

    # --- Collect all descriptors ---
    return {
        "smiles": smiles,
        "charge": charge,
        "spin_multiplicity": spin_multiplicity,
        "hf_energy": hf_energy,
        "vqe_energy": vqe_energy,
        "correlation_energy": hf_energy - vqe_energy,
        "n_qubits": qubit_op.num_qubits,
        "homo_energy": homo_energy,
        "lumo_energy": lumo_energy,
        "homo_lumo_gap": lumo_energy - homo_energy,
        "dipole_moment": dipole_moment,
    }

# ============================================================
# Example Run
# ============================================================
if __name__ == "__main__":
    hydroxide_results = compute_quantum_descriptors(smiles="[OH-]")
    print("\n--- Final Descriptors for [OH-] ---")
    for key, value in hydroxide_results.items():
        print(f"{key:<20}: {value:.6f}" if isinstance(value, float) else f"{key:<20}: {value}")
    print("-----------------------------------")


[•] Processing molecule: [OH-]
[i] Auto-detected Charge: -1, Auto-detected Spin: 1
[+] Hartree–Fock energy: -74.060416 Hartree
[+] VQE ground-state energy: -0.629590 Hartree
converged SCF energy = -74.0604157013592

--- Final Descriptors for [OH-] ---
smiles              : [OH-]
charge              : -1
spin_multiplicity   : 1
hf_energy           : -74.060416
vqe_energy          : -0.629590
correlation_energy  : -73.430825
n_qubits            : 4
homo_energy         : 0.247616
lumo_energy         : 1.222037
homo_lumo_gap       : 0.974422
dipole_moment       : 0.804211
-----------------------------------


In [33]:
import pandas as pd
from tqdm import tqdm

hf_energies = []
vqe_energies = []
corr_energies = []
n_qubits_list = []
homo_list = []
lumo_list = []
gap_list = []
dipole_list = []

for smiles in tqdm(df["mol"], desc="Computing quantum descriptors"):
    try:
        descriptors = compute_quantum_descriptors(smiles, active_electrons=2, active_orbitals=2)
        if descriptors: # Check if descriptors were successfully computed (not None)
            hf_energies.append(descriptors["hf_energy"])
            vqe_energies.append(descriptors["vqe_energy"])
            corr_energies.append(descriptors["correlation_energy"])
            n_qubits_list.append(descriptors["n_qubits"])
            homo_list.append(descriptors["homo_energy"])
            lumo_list.append(descriptors["lumo_energy"])
            gap_list.append(descriptors["homo_lumo_gap"])
            dipole_list.append(descriptors["dipole_moment"])
        else:
            hf_energies.append(None)
            vqe_energies.append(None)
            corr_energies.append(None)
            n_qubits_list.append(None)
            homo_list.append(None)
            lumo_list.append(None)
            gap_list.append(None)
            dipole_list.append(None)

    except Exception as e:
        print(f"[!] Error processing molecule {smiles}: {e}")
        # Append None if an unexpected error occurs
        hf_energies.append(None)
        vqe_energies.append(None)
        corr_energies.append(None)
        n_qubits_list.append(None)
        homo_list.append(None)
        lumo_list.append(None)
        gap_list.append(None)
        dipole_list.append(None)

df["hf_energy"] = hf_energies
df["vqe_energy"] = vqe_energies
df["correlation_energy"] = corr_energies
df["n_qubits"] = n_qubits_list
df["homo_energy"] = homo_list
df["lumo_energy"] = lumo_list
df["homo_lumo_gap"] = gap_list
df["dipole_moment"] = dipole_list
#df["polarizability"] = polarizability_list

print("\n Quantum descriptors added successfully to df!")
print(df.head())

df.to_csv("bace_with_quantum_features.csv", index=False)
from google.colab import files
files.download("bace_with_subset.csv")
print("Saved as bace_with_quantum_features.csv")

Computing quantum descriptors:   0%|          | 0/9 [00:00<?, ?it/s]


[•] Processing molecule: n1cccc(NCc2ccccc2)c1N
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -617.581672 Hartree
[+] VQE ground-state energy: -0.682487 Hartree
converged SCF energy = -617.581672336509


Computing quantum descriptors:  11%|█         | 1/9 [01:10<09:26, 70.87s/it]


[•] Processing molecule: Fc1ccc(cc1)CC1CC[NH2+]CC1
[i] Auto-detected Charge: 1, Auto-detected Spin: 1
[+] Hartree–Fock energy: -610.436124 Hartree
[+] VQE ground-state energy: -1.011170 Hartree
converged SCF energy = -610.436123922738


Computing quantum descriptors:  22%|██▏       | 2/9 [02:21<08:15, 70.77s/it]

...


[•] Processing molecule: Oc1ccc(cc1)CC[NH3+]
[i] Auto-detected Charge: 1, Auto-detected Spin: 1
[+] Hartree–Fock energy: -433.621565 Hartree
[+] VQE ground-state energy: -0.997360 Hartree
converged SCF energy = -433.62156537467


Computing quantum descriptors:  89%|████████▉ | 8/9 [06:16<00:42, 42.44s/it]


[•] Processing molecule: Clc1cc2nc([nH]c2cc1)N
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -881.091355 Hartree
[+] VQE ground-state energy: -0.690594 Hartree
converged SCF energy = -881.091355419743


Computing quantum descriptors: 100%|██████████| 9/9 [06:51<00:00, 45.68s/it]


 Quantum descriptors added successfully to subset_df!
                           mol  Class  \
192      n1cccc(NCc2ccccc2)c1N      0   
194  Fc1ccc(cc1)CC1CC[NH2+]CC1      0   
200          n1ccc2c(cccc2)c1N      0   
201          n1c2c(ccc1N)cccc2      0   
203       O=C1NC(=NC(=C1)CCC)N      0   

                                              features   hf_energy  \
192  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -617.581672   
194  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -610.436124   
200  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -448.735581   
201  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -448.738868   
203  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -503.251281   

     vqe_energy  correlation_energy  n_qubits  homo_energy  lumo_energy  \
192   -0.682487         -616.899185         4    -0.203192     0.251207   
194   -1.011170         -609.424954         4    -0.346853     0.169082   
200   -0.486979         -448.248602         4    -0.

## **Training ML Model Using Classically Obtained Features Only**

In [76]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
pca.fit(X)
X_pca = pca.transform(X)
print("X_train shape after PCA:", X_pca.shape)

X_train shape after PCA: (9, 7)


In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=2)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (7, 7)
X_test shape: (2, 7)
y_train shape: (7,)
y_test shape: (2,)


In [80]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

print("Random Forest Regressor model trained successfully!")

Random Forest Regressor model trained successfully!


In [82]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Calculate evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the evaluation metrics
print(f"R-squared (R2) Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

R-squared (R2) Score: 0.0000
Mean Absolute Error (MAE): 0.5050
Mean Squared Error (MSE): 0.3102
Root Mean Squared Error (RMSE): 0.5570


In [68]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)
baseline_score = dummy.score(X_test, y_test)
print(f"Baseline R²: {baseline_score}")

Baseline R²: -0.18367346938775508


## **Training ML Model Using the Quantum Features Also**

In [83]:
X_=np.array(pd.DataFrame(df,columns=["hf_energy","vqe_energy","homo_energy","lumo_energy","dipole_moment"]))
X_Q=np.hstack((X_pca,X_))

from sklearn.model_selection import train_test_split
X_train_Q, X_test_Q, y_train_Q, y_test_Q = train_test_split(X_Q, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train_Q.shape)
print("X_test shape:", X_test_Q.shape)

X_train shape: (7, 12)
X_test shape: (2, 12)


In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_Q, y_train_Q)

print("Random Forest Regressor model trained successfully!")

Random Forest Regressor model trained successfully!


In [85]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Make predictions on the test set
y_pred = rf_model.predict(X_test_Q)

# Calculate evaluation metrics
r2 = r2_score(y_test_Q, y_pred)
mae = mean_absolute_error(y_test_Q, y_pred)
mse = mean_squared_error(y_test_Q, y_pred)
rmse = np.sqrt(mse)

# Print the evaluation metrics
print(f"R-squared (R2) Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

R-squared (R2) Score: 0.0636
Mean Absolute Error (MAE): 0.4600
Mean Squared Error (MSE): 0.2341
Root Mean Squared Error (RMSE): 0.4838


In [87]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train_Q, y_train_Q)
baseline_score = dummy.score(X_test_Q, y_test_Q)
print(f"Baseline R²: {baseline_score}")

Baseline R²: -0.18367346938775508
